<a href="https://colab.research.google.com/github/Anthlis/pyb_scratchpad/blob/master/bite92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pytest pytest-sugar 

  Created wheel for pytest-sugar: filename=pytest_sugar-0.9.4-py2.py3-none-any.whl size=8969 sha256=3db8c778f90d8eafbd3e9c300a83f91f48cee2535538d056e23a44f91b34e5cc
  Stored in directory: /root/.cache/pip/wheels/72/ac/7f/55050ee786fa4a2bb2d05dea0696eab826ff1d5b8a7dbd5883
Successfully built pytest-sugar


In [1]:
%%file test_humanize_date.py

from datetime import timedelta

import pytest

from humanize_date import pretty_date, NOW


def n_days_ago_str(days):
    return (NOW - timedelta(days=days)).strftime('%m/%d/%y')


@pytest.mark.parametrize("arg, expected", [
    (NOW - timedelta(seconds=2), 'just now'),
    (NOW - timedelta(seconds=9), 'just now'),
    (NOW - timedelta(seconds=10), '10 seconds ago'),
    (NOW - timedelta(seconds=59), '59 seconds ago'),
    (NOW - timedelta(minutes=1), 'a minute ago'),
    (NOW - timedelta(minutes=1, seconds=40), 'a minute ago'),
    (NOW - timedelta(minutes=2), '2 minutes ago'),
    (NOW - timedelta(minutes=59), '59 minutes ago'),
    (NOW - timedelta(hours=1), 'an hour ago'),
    (NOW - timedelta(hours=2), '2 hours ago'),
    (NOW - timedelta(hours=23), '23 hours ago'),
    (NOW - timedelta(hours=24), 'yesterday'),
    (NOW - timedelta(hours=47), 'yesterday'),
    (NOW - timedelta(days=1), 'yesterday'),
    (NOW - timedelta(days=2), n_days_ago_str(2)),
    (NOW - timedelta(days=7), n_days_ago_str(7)),
    (NOW - timedelta(days=100), n_days_ago_str(100)),
    (NOW - timedelta(days=365), n_days_ago_str(365)),
])
def test_pretty_date(arg, expected):
    assert pretty_date(arg) == expected


def test_input_variable_of_wrong_type():
    with pytest.raises(ValueError):
        pretty_date(123)


def test_input_variable_future_date():
    with pytest.raises(ValueError):
        pretty_date(NOW + timedelta(days=1))

Writing test_humanize_date.py


In [6]:
%%file humanize_date.py

from collections import namedtuple
from datetime import datetime, timedelta

TimeOffset = namedtuple('TimeOffset', 'offset date_str divider')

NOW = datetime.now()
MINUTE, HOUR, DAY = 60, 60*60, 24*60*60
TIME_OFFSETS = (
    TimeOffset(10, 'just now', None),
    TimeOffset(MINUTE, '{} seconds ago', None),
    TimeOffset(2*MINUTE, 'a minute ago', None),
    TimeOffset(HOUR, '{} minutes ago', MINUTE),
    TimeOffset(2*HOUR, 'an hour ago', None),
    TimeOffset(DAY, '{} hours ago', HOUR),
    TimeOffset(2*DAY, 'yesterday', None),
)


def pretty_date(date):
    """Receives a datetime object and converts/returns a readable string
       using TIME_OFFSETS"""
    if type(date) is not datetime or date > NOW:
        raise ValueError
    time_in_seconds = (NOW - date).days * DAY + (NOW - date).seconds
    for time in TIME_OFFSETS:
        if time_in_seconds < time.offset:
            if time.divider == None:
                return time.date_str.format(time_in_seconds)
            else:
                return time.date_str.format(time_in_seconds//time.divider)
    return date.strftime('%m/%d/%y')


print(pretty_date((NOW - timedelta(seconds=15))))

Overwriting humanize_date.py


In [10]:
!python humanize_date.py

15 seconds ago


In [4]:
!python -m pytest test_humanize_date.py --verbose

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 20 items                                                             

test_humanize_date.py::test_pretty_date[arg0-just now] PASSED            [  5%]
test_humanize_date.py::test_pretty_date[arg1-just now] PASSED            [ 10%]
test_humanize_date.py::test_pretty_date[arg2-10 seconds ago] PASSED      [ 15%]
test_humanize_date.py::test_pretty_date[arg3-59 seconds ago] PASSED      [ 20%]
test_humanize_date.py::test_pretty_date[arg4-a minute ago] PASSED        [ 25%]
test_humanize_date.py::test_pretty_date[arg5-a minute ago] PASSED        [ 30%]
test_humanize_date.py::test_pretty_date[arg6-2 minutes ago] PASSED       [ 35%]
test_humanize_date.py::test_pretty_date[arg7-59 minutes ago] PASSED      [ 40%]
test_humanize_date.py::test_pret